In [ ]:
#Pregunta 1

In [1]:
#Pregunta 2
#Item A
import pandas as pd
import os
os.getcwd() # Ver donde estamos ubicados
#os.chdir("/Users/bautistagoulu/Desktop/Big Data") # Ubicarnos en la carpeta con todos los archivos de esta tutorial
os.chdir("/Users/francodeangelis/Documents/Udesa/cuarto /Big data /BigDataUdeSA/ejercico 1/EPH_usu_1er_Trim_2024_xlsx")
df24 = pd.read_excel("usu_individual_T124.xlsx")

In [ ]:
# Filtro los datos
df24=df24[df24[ "REGION" ]== 43]
df24=df24[(df24[ "AGLOMERADO" ]==36)|(df24[ "AGLOMERADO" ]==13)|(df24[ "AGLOMERADO" ]==30)]
print(df24)

In [ ]:
os.getcwd() # Ver donde estamos ubicados
#os.chdir("/Users/bautistagoulu/Desktop/Big Data")# Ubicarnos en la carpeta con todos los archivos de esta tutorial
os.chdir("/Users/francodeangelis/Documents/Udesa/cuarto /Big data /BigDataUdeSA/ejercico 1/t104_dta")
df04 = pd.read_stata("Individual_t104.dta")
df04.head(2)

In [ ]:
# Filtro los datos  
df04=df04[df04['region']=="Pampeana"]  
df04=df04[(df04['aglomerado']=="Río Cuarto") | (df04['aglomerado']=="Gran Cordoba") | (df04['aglomerado']=="Santa Rosa")]  
print(df04)

In [ ]:

# ---- Normalizar nombres de columnas ----
df24.columns = df24.columns.str.upper().str.strip()  # Convertir a mayúsculas y eliminar espacios
df04.columns = df04.columns.str.upper().str.strip()  # Convertir a mayúsculas y eliminar espacios

In [ ]:
# Unifico los DF
df = pd.concat([df24, df04])
print(df.shape)
print(df)
df.to_excel("Tabla_acortada.xlsx", index = False)

In [ ]:
#Item B

import seaborn as sns
import matplotlib.pyplot as plt

variables_interes=['NIVEL_ED','ESTADO','CAT_OCUP','CAT_INAC','PP02C1','PP02C2','PP02C3','PP02C4','PP02C5','PP02C6']
df_interes = df[variables_interes]

missing_data = df_interes.isnull() # Nuevo DF donde cada celda contiene un valor True si el dato está faltante (es NaN), o False si el dato está presente. Este nuevo DataFrame missing_data es el que usaremos para crear el heatmap.

plt.figure(figsize=(8, 6))  # Establecemos el tamaño de la figura del gráfico
sns.heatmap(missing_data, annot=False, cmap='coolwarm_r', cbar_kws={'label': 'Percentage of Non-null Data'})
plt.title("Missing Data for Selected Variables")  # Título del gráfico
plt.xlabel("Variables")  # Etiqueta del eje X (columnas)
plt.ylabel("Observations")  # Etiqueta del eje Y (filas)
plt.show()  # Muestra el gráfico

In [ ]:
#Item C
import numpy as np

#Queremos reemplazar por nan los valores sin sentido. 

#Consideraciones:

#P21, P47T, ITF, IPCF (Ingresos): Valores negativos son inconsistentes salvo excepciones justificadas.
ingresos_vars = ['P21', 'P47T', 'ITF', 'IPCF']
for var in ingresos_vars:
    df[var] = pd.to_numeric(df[var], errors='coerce')
    df.loc[df[var] < 0, var] = np.nan
     
#PP3E_TOT, PP3F_TOT (Horas trabajadas): Valores negativos o superiores a 100 por semana no son realistas.
horas_vars = ['PP3E_TOT', 'PP3F_TOT']
for var in horas_vars:
    df.loc[(df[var] < 0) | (df[var] > 100), var] = np.nan
    
#CH14 (Último año aprobado): Valores fuera del rango 00-09 (excepto casos especiales como 98 o 99).
df['CH14'] = pd.to_numeric(df['CH14'], errors='coerce')
df.loc[~df['CH14'].between(0, 9), 'CH14'] = np.nan

#PP04B3_MES, PP04B3_ANO, PP04B3_DIA (Tiempo trabajado): Fechas imposibles como meses fuera de 1-12 o días fuera de 1-31
df['PP04B3_MES'] = pd.to_numeric(df['PP04B3_MES'], errors='coerce')
df['PP04B3_DIA'] = pd.to_numeric(df['PP04B3_DIA'], errors='coerce')
df.loc[(df['PP04B3_MES'] < 1) | (df['PP04B3_MES'] > 12), 'PP04B3_MES'] = np.nan
df.loc[(df['PP04B3_DIA'] < 1) | (df['PP04B3_DIA'] > 31), 'PP04B3_DIA'] = np.nan

# Verificar cuántos valores se convirtieron en NaN
print(df.isnull().sum())  #muestra el conteo de valores faltantes (NaN) por cada variable en el DataFrame

In [ ]:
#ITEM D

#En 2004: La variable TRIMESTRE tiene valores como:

#1er. trimestre, 2do. trimestre, 3er. trimestre, 4to. trimestre 

#En 2024: La variable TRIMESTRE tiene valores numéricos:

#1 = Primer trimestre

#2 = Segundo trimestre

#3 = Tercer trimestre

#4 = Cuarto trimestre

# Homogenizamos la variable TRIMESTRE de 2004 (texto) a numérico (1, 2, 3, 4)
df['TRIMESTRE'] = df['TRIMESTRE'].replace({
    '1er. trimestre': 1,
    '2do. trimestre': 2,
    '3er. trimestre': 3,
    '4to. trimestre': 4
})

# Verificamos la homogenización
print(df[['TRIMESTRE']].head())

In [ ]:
#Lo mismo sucede con los valores de REGION, en 2004 estan escritos y en 2024 se les asigna un numero.

# Asegurarnos de que la columna REGION sea de tipo string antes de hacer el reemplazo
df['REGION'] = df['REGION'].astype(str)

# Reemplazar los valores de REGION (de nombre a código numérico)
df['REGION'] = df['REGION'].replace({
    'Gran Buenos Aires': 1,
    'NOA': 40,
    'NEA': 41,
    'Cuyo': 42,
    'Pampeana': 43,
    'Patagónica': 44
})

# Verificamos la homogenización de los valores en REGION
print(df[['REGION']].head())


In [ ]:
#La variable AGLOMERADO también presenta una diferencia en la forma en que se codifica entre 2004 y 2024.
# Asegurarse de que la columna AGLOMERADO sea de tipo string antes de hacer el reemplazo
df['AGLOMERADO'] = df['AGLOMERADO'].astype(str)

# Reemplazar los valores de AGLOMERADO de 2004 (nombre) a código numérico
df['AGLOMERADO'] = df['AGLOMERADO'].replace({
    'Gran La Plata': 2,
    'Bahía Blanca - Cerri': 3,
    'Gran Rosario': 4,
    'Gran Santa Fé': 5,
    'Gran Paraná': 6,
    'Posadas': 7,
    'Gran Resistencia': 8,
    'Comodoro Rivadavia - Rada Tilly': 9,
    'Gran Mendoza': 10,
    'Corrientes': 12,
    'Gran Córdoba': 13,
    'Concordia': 14,
    'Formosa': 15,
    'Neuquén – Plottier': 17,
    'Santiago del Estero - La Banda': 18,
    'Jujuy - Palpalá': 19,
    'Río Gallegos': 20,
    'Gran Catamarca': 22,
    'Gran Salta': 23,
    'La Rioja': 25,
    'Gran San Luis': 26,
    'Gran San Juan': 27,
    'Gran Tucumán - Tafí Viejo': 29,
    'Santa Rosa – Toay': 30,
    'Ushuaia - Río Grande': 31,
    'Ciudad Autónoma de Buenos Aires': 32,
    'Partidos del GBA': 33,
    'Mar del Plata': 34,
    'Río Cuarto': 36,
    'San Nicolás – Villa Constitución': 38,
    'Rawson – Trelew': 91,
    'Viedma – Carmen de Patagones': 93
})

# Verificamos la homogenización de la variable
print(df[['AGLOMERADO']].head())

In [ ]:
# Seguimos con la variable CH04 (sexo)

import numpy as np

# Asegurarnos de que la columna CH04 sea de tipo string antes de hacer el reemplazo
df['CH04'] = df['CH04'].astype(str)

# Reemplazar los valores de CH04 de 2004 (texto) a valores numéricos
df['CH04'] = df['CH04'].replace({
    'Varón': 1,
    'Mujer': 2
})

# Convertir los valores a numéricos (eliminando cualquier valor no numérico)
df['CH04'] = pd.to_numeric(df['CH04'], errors='coerce')  # Coerce convierte valores no numéricos a NaN

# Convertir los valores de CH04 a enteros (y manejar los NaN)
df['CH04'] = df['CH04'].fillna(1).astype(int)  # Llenamos NaN con 1 (o el valor adecuado) y luego convertimos a int

# Verificamos la homogenización
print(df[['CH04']].head())

In [ ]:
#En la columna CH006 cuando la edad esta entre 0 y 1, en 2024 aparece -1, y en 2004 dice "Menos de 1 año", homogenizemos para que diga -1
# Asegurarnos de que la columna CH06 sea de tipo string antes de hacer el reemplazo
df['CH06'] = df['CH06'].astype(str)

# Reemplazar "Menos de 1 año" por -1 en 2004
df['CH06'] = df['CH06'].replace({
    'Menos de 1 año': -1
})

# Convertir los valores a numérico, forzando los errores a NaN
df['CH06'] = pd.to_numeric(df['CH06'], errors='coerce')

# Llenar los NaN (si los hay) con un valor adecuado (por ejemplo, 0 o el valor adecuado)
df['CH06'] = df['CH06'].fillna(-1).astype(int)

# Verificamos la homogenización
print(df[['CH06']].head())


In [ ]:
# Seguimos con la variable CH07 (Estado civil)
# Asegurarnos de que la columna CH07 sea de tipo string antes de hacer el reemplazo
df['CH07'] = df['CH07'].astype(str)

# Reemplazar los valores de CH07 de 2004 (texto) a valores numéricos
df['CH07'] = df['CH07'].replace({
    'unido?': 5,
    'casado?': 1,
    'separado/a o divorciado/a?': 3,
    'viudo/a?': 4,
    'soltero/a?': 2
})

# Convertir los valores de CH07 a numérico, forzando los errores a NaN
df['CH07'] = pd.to_numeric(df['CH07'], errors='coerce')

# Llenar los NaN (si los hay) con 1 o el valor adecuado
df['CH07'] = df['CH07'].fillna(1).astype(int)  # Si hay NaN, se reemplaza por 1 (o el valor adecuado)

# Verificamos la homogenización
print(df[['CH07']].head())


In [ ]:
#Lo mismo para variable CH08 (Cobertura medica)
# Asegurarnos de que la columna CH08 sea de tipo string antes de hacer el reemplazo
df['CH08'] = df['CH08'].astype(str)

# Reemplazar los valores de CH08 de 2004 (texto) a valores numéricos
df['CH08'] = df['CH08'].replace({
    'Obra social (incluye PAMI)': 1,
    'Mutual / prepaga / servicio de emergencia': 2,
    'Planes y seguros públicos': 3,
    'No paga ni le descuentan': 4,
    'Ns/Nr': 9,
    'Obra social y mutual / prepaga / servicio de emergencia': 12,
    'Obra social y planes y seguros públicos': 13,
    'Mutual / prepaga / servicio de emergencia / Planes y seguros públicos': 23,
    'Obra social, mutual / prepaga / servicio de emergencia y planes y seguros públicos': 123
})

# Convertir los valores a numérico, forzando los errores a NaN
df['CH08'] = pd.to_numeric(df['CH08'], errors='coerce')

# Llenar los NaN (si los hay) con un valor adecuado (por ejemplo, 1)
df['CH08'] = df['CH08'].fillna(1).astype(int)

# Verificamos la homogenización
print(df[['CH08']].head())

In [ ]:
# Seguimos con la columna de NIVEL_ED

#Asegurarnos de que la columna NIVEL_ED sea de tipo string antes de hacer el reemplazo
df['NIVEL_ED'] = df['NIVEL_ED'].astype(str)

# Reemplazar los valores de NIVEL_ED de 2004 (texto) a valores numéricos
df['NIVEL_ED'] = df['NIVEL_ED'].replace({
    'Primario incompleto (incluye educación especial)': 1,
    'Primario completo': 2,
    'Secundario incompleto': 3,
    'Secundario completo': 4,
    'Superior universitario incompleto': 5,
    'Superior universitario completo': 6,
    'Sin instrucción': 7,
    'Ns/Nr': 9
})

# Convertir los valores a numérico, forzando los errores a NaN
df['NIVEL_ED'] = pd.to_numeric(df['NIVEL_ED'], errors='coerce')

# Llenar los NaN (si los hay) con un valor adecuado (por ejemplo, 1 o el valor adecuado)
df['NIVEL_ED'] = df['NIVEL_ED'].fillna(1).astype(int)

# Verificamos la homogenización
print(df[['NIVEL_ED']].head())

In [ ]:
#Seguimos con la variable ESTADO

# Asegurarnos de que la columna ESTADO sea de tipo string antes de hacer el reemplazo
df['ESTADO'] = df['ESTADO'].astype(str)

# Reemplazar los valores de ESTADO de 2004 (texto) a valores numéricos
df['ESTADO'] = df['ESTADO'].replace({
    'Entrevista individual no realizada (no respuesta al cuestionario individual)': 0,
    'Ocupado': 1,
    'Desocupado': 2,
    'Inactivo': 3,
    'Menor de 10 años': 4
})

# Convertir los valores a numérico, forzando los errores a NaN
df['ESTADO'] = pd.to_numeric(df['ESTADO'], errors='coerce')

# Llenar los NaN (si los hay) con un valor adecuado (por ejemplo, 0)
df['ESTADO'] = df['ESTADO'].fillna(0).astype(int)

# Verificamos la homogenización
print(df[['ESTADO']].head())

In [ ]:
#Seguimos con la variable ESTADO

# Asegurarnos de que la columna ESTADO sea de tipo string antes de hacer el reemplazo
df['ESTADO'] = df['ESTADO'].astype(str)

# Reemplazar los valores de ESTADO de 2004 (texto) a valores numéricos
df['ESTADO'] = df['ESTADO'].replace({
    'Entrevista individual no realizada (no respuesta al cuestionario individual)': 0,
    'Ocupado': 1,
    'Desocupado': 2,
    'Inactivo': 3,
    'Menor de 10 años': 4
})

# Convertir los valores a numérico, forzando los errores a NaN
df['ESTADO'] = pd.to_numeric(df['ESTADO'], errors='coerce')

# Llenar los NaN (si los hay) con un valor adecuado (por ejemplo, 0)
df['ESTADO'] = df['ESTADO'].fillna(0).astype(int)

# Verificamos la homogenización
print(df[['ESTADO']].head())

In [ ]:
#Punto 3
import matplotlib.pyplot as plt   
import seaborn as sns
print(sexo_counts.shape)
# Contar la cantidad de personas por sexo y año
sexo_counts = df.groupby(["ANO4", "CH04"]).size().reset_index(name="CANTIDAD")

# Crear el gráfico de barras
plt.figure(figsize=(8, 5))
sns.barplot(data=sexo_counts, x="ANO4", y="CANTIDAD", hue="CH04", palette="viridis")

# Agregar etiquetas y título
plt.xlabel("Año")
plt.ylabel("Cantidad de personas")
plt.title("Composición por sexo en 2004 y 2024")
plt.legend(title="Sexo")
plt.show()


In [ ]:
#Punto 4

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Variables de interés
variables_interes = ["CH04", "CH06", "CH07", "CH08", "NIVEL_ED", "ESTADO", "CAT_INAC", "IPCF"]

# Convertir variables a numéricas (si hay valores no numéricos, los pone en NaN)
df_corr = df[variables_interes].apply(pd.to_numeric, errors="coerce")

In [ ]:
df_corr_2004 = df_corr[df["ANO4"] == 2004]
df_corr_2024 = df_corr[df["ANO4"] == 2024]
corr_2004 = df_corr_2004.corr()
corr_2024 = df_corr_2024.corr()


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Matriz de correlación 2004
sns.heatmap(corr_2004, annot=True, fmt=".2f", cmap="coolwarm", ax=axes[0])
axes[0].set_title("Matriz de Correlación - 2004")

# Matriz de correlación 2024
sns.heatmap(corr_2024, annot=True, fmt=".2f", cmap="coolwarm", ax=axes[1])
axes[1].set_title("Matriz de Correlación - 2024")

plt.show()


In [ ]:
# Configurar el tamaño de las figuras
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Matriz de correlación 2004
sns.heatmap(corr_2004, annot=True, fmt=".2f", cmap="coolwarm", ax=axes[0])
axes[0].set_title("Matriz de Correlación - 2004")

# Matriz de correlación 2024
sns.heatmap(corr_2024, annot=True, fmt=".2f", cmap="coolwarm", ax=axes[1])
axes[1].set_title("Matriz de Correlación - 2024")

plt.show()

In [ ]:
#Punto 5
# Verificar valores únicos en "ESTADO" y "CAT_INAC" para evitar errores
print("Valores en ESTADO:", df["ESTADO"].unique())
print("Valores en CAT_INAC:", df["CAT_INAC"].unique())

# Contar desocupados
desocupados = df[df["ESTADO"] == "Desocupado"].shape[0]

# Contar inactivos
inactivos = df[df["CAT_INAC"] == "Inactivo"].shape[0]

# Calcular media del ingreso per cápita familiar (IPCF) según estado laboral
ipcf_media = df.groupby("ESTADO")["IPCF"].mean()

# Mostrar resultados
print(f"Cantidad de desocupados: {desocupados}")
print(f"Cantidad de inactivos: {inactivos}")
print("\nMedia del IPCF según estado laboral:")
print(ipcf_media)


In [ ]:
#Punto 6(Este lo hizo Franco,ver lo que hizo Gian)
print(df["ESTADO"].value_counts())

respondieron = df[df["ESTADO"].notna()]  # Filtra los que tienen un valor válido en ESTADO
norespondieron = df[df["ESTADO"].isna()]  # Filtra los que tienen NaN en ESTADO

respondieron.to_excel("respondieron.xlsx", index=False)
norespondieron.to_excel("norespondieron.xlsx", index=False)

print(f"Personas que respondieron: {respondieron.shape[0]}")
print(f"Personas que NO respondieron: {norespondieron.shape[0]}")

In [ ]:
#Punto 9 Este lo hizo Franco,ver lo que hizo Gian)
#A
import seaborn as sns
import matplotlib.pyplot as plt

# Agrupar por año y nivel educativo para calcular la proporción de desocupados
desocupados_educacion = respondieron.groupby(["ANO4", "NIVEL_ED"])["desocupado"].mean().reset_index()

# Graficar comparación
plt.figure(figsize=(10, 6))
sns.barplot(data=desocupados_educacion, x="NIVEL_ED", y="desocupado", hue="ANO4", palette="viridis")

plt.xlabel("Nivel Educativo")
plt.ylabel("Proporción de Desocupados")
plt.title("Desocupación por Nivel Educativo en 2004 vs 2024")
plt.legend(title="Año")

plt.xticks(rotation=45)  # Rotar etiquetas para mejor visualización
plt.show()
#b
# Crear categorías de edad en grupos de 10 años
bins = [0, 10, 20, 30, 40, 50, 60, 70, 100]  # Definir los rangos de edad
labels = ["0-9", "10-19", "20-29", "30-39", "40-49", "50-59", "60-69", "70+"]
respondieron["edad_grupo"] = pd.cut(respondieron["CH06"], bins=bins, labels=labels, right=False)

# Calcular proporción de desocupados por edad agrupada
desocupados_edad = respondieron.groupby(["ANO4", "edad_grupo"])["desocupado"].mean().reset_index()

# Graficar comparación
plt.figure(figsize=(10, 6))
sns.barplot(data=desocupados_edad, x="edad_grupo", y="desocupado", hue="ANO4", palette="magma")

plt.xlabel("Grupo de Edad")
plt.ylabel("Proporción de Desocupados")
plt.title("Desocupación por Grupo de Edad en 2004 vs 2024")
plt.legend(title="Año")

plt.show()
#c
# Calcular proporción de desocupados por sexo
desocupados_sexo = respondieron.groupby(["ANO4", "CH04"])["desocupado"].mean().reset_index()

# Graficar comparación
plt.figure(figsize=(8, 5))
sns.barplot(data=desocupados_sexo, x="CH04", y="desocupado", hue="ANO4", palette="coolwarm")

plt.xlabel("Sexo")
plt.ylabel("Proporción de Desocupados")
plt.title("Desocupación por Sexo en 2004 vs 2024")
plt.legend(title="Año")

plt.show()